In [3]:
import impala.dbapi
import impala.util
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
from datetime import date
import matplotlib.gridspec as gridspec
today = date.today()
import matplotlib.ticker as ticker
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(font_scale=1.5, palette='bright')
# widen the cell 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#imp_connection = impala.dbapi.connect(host='impala-lb-main.data.houzz.net', port=21050)

imp_connection = impala.dbapi.connect(host='impala-lb.data.houzz.net', port=21050)
imp_cursor = imp_connection.cursor()
pd.set_option('max_colwidth', 200)
def impala_run(q):
    return pd.read_sql(q, imp_connection)

import plotly.express as px
import plotly.graph_objects as go


import impala.util
import impala.dbapi
from impala.dbapi import connect
import pandas as pd
imp_connection = connect(host='impala-lb.data.houzz.net', port=21050)


imp_connection = impala.dbapi.connect(host='impala-lb-alpha.data.houzz.net', port=21050)
imp_cursor = imp_connection.cursor()
def impala_alpha_run(q):
    return pd.read_sql(q, imp_connection)

imp_connection = impala.dbapi.connect(host='impala-lb-main.data.houzz.net', port=21050)
imp_cursor = imp_connection.cursor()
def impala_main_run(q):
    return pd.read_sql(q, imp_connection)


from pyhive import presto
def get_presto_conn():
    return presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086)
conn_pres = get_presto_conn() # establish the connection
def presto_run(q):
    return pd.read_sql(q, conn_pres)

# Statistical tests for experiment analysis

In [ ]:
# statistical tests for aggregated metrics
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats
def z_test_prop_agg(n_ctr, n_exp, x_ctr, x_exp):
    p_ctr = x_ctr/n_ctr
    p_exp = x_exp/n_exp
    p_pool = (x_ctr + x_exp)/(n_ctr + n_exp)
    se_pool = np.sqrt(p_pool * (1-p_pool) * (1/n_ctr + 1/n_exp))
    z_score = (p_exp - p_ctr)/se_pool
    pval = stats.norm.sf(abs(z_score))*2
    diff = p_exp - p_ctr
    lower_CI = diff-1.96*se_pool    # 2.58 for 0.01 significance level, 1.96 for 0.05
    higher_CI = diff+1.96*se_pool     # 2.58 for 0.01 significance level, 1.96 for 0.05
    diff_perc = diff/(p_ctr+1e-12) * 100
    lower_CI_perc = lower_CI/(p_ctr+1e-12) * 100
    higher_CI_perc = higher_CI/(p_ctr+1e-12) * 100
    return n_ctr, n_exp, p_ctr, p_exp, diff, lower_CI, higher_CI, diff_perc, lower_CI_perc, higher_CI_perc, pval 
def t_test_mean_agg(n_ctr, n_exp, mean_ctr, mean_exp, std_ctr, std_exp):  
    diff = mean_exp - mean_ctr    
    std_pool = np.sqrt(((n_ctr-1)*std_ctr*std_ctr + (n_exp-1)*std_exp*std_exp)/(n_ctr+n_exp-2))
    se_pool = std_pool * np.sqrt(1/n_ctr + 1/n_exp)
    dof = n_ctr+n_exp-2    
    t_score = diff / se_pool
    pval = stats.t.sf(np.abs(t_score), dof)*2
    lower_CI = diff-1.96*se_pool    # 2.58 for 0.01 significance level, 1.96 for 0.05
    higher_CI = diff+1.96*se_pool  # 2.58 for 0.01 significance level, 1.96 for 0.05
    diff_perc = diff/(mean_ctr+1e-12) * 100
    lower_CI_perc = lower_CI/(mean_ctr+1e-12) * 100
    higher_CI_perc = higher_CI/(mean_ctr+1e-12) * 100
    return n_ctr, n_exp, mean_ctr, mean_exp, diff, lower_CI, higher_CI, diff_perc, lower_CI_perc, higher_CI_perc, pval 
# Highlight if difference is significant
def highlight_xlong(s):
    if s['% diff'] == 'N/A':
        return ['color:black']*3    
    elif float(s['% diff'].split('p=')[1]) < 0.05 and float(s['% diff'].split('%')[0]) < 0:
        return ['color:black']*2 + ['color: red']*1    
    elif float(s['% diff'].split('p=')[1]) < 0.05 and float(s['% diff'].split('%')[0]) > 0:
        return ['color:black']*2 + ['color: green']*1    
    else:
        return ['color:black']*3 
def highlight_xshort(s):
    if s['% diff'] == 'N/A':
        return ['color:black']*3    
    elif s['% diff'].split('%')[1].strip().strip('(').strip(')') == 'sig' and float(s['% diff'].split('%')[0]) < 0:
        return ['color:black']*2 + ['color: red']*1    
    elif s['% diff'].split('%')[1].strip().strip('(').strip(')') == 'sig' and float(s['% diff'].split('%')[0]) > 0:
        return ['color:black']*2 + ['color: green']*1    
    else:
        return ['color:black']*3  
def output_format(results, ctr_name, exp_name, metric_type, details):
    N_ctr, N_exp, metric_ctr, metric_exp, diff, lower_CI, higher_CI, diff_perc, lower_CI_perc, higher_CI_perc, pval = results
    if diff_perc > 10000:
        diff_perc = np.inf
    elif diff_perc < -10000:
        diff_perc = -np.inf
    if lower_CI_perc > 10000:
        lower_CI_perc = np.inf
    elif lower_CI_perc < -10000:
        lower_CI_perc = -np.inf
    if higher_CI_perc > 10000:
        higher_CI_perc = np.inf      
    elif higher_CI_perc < -10000:
        higher_CI_perc = -np.inf 
    output_dict = dict()  
    output_dict[ctr_name] = '{0:.4f}'.format(metric_ctr) if metric_type == 'mean' \
                                            else '{0:.3f}%'.format(metric_ctr*100) if metric_type == 'proportion' \
                                            else 'error'
    output_dict[exp_name] = '{0:.4f}'.format(metric_exp) if metric_type == 'mean' \
                                            else '{0:.3f}%'.format(metric_exp*100) if metric_type == 'proportion' \
                                            else 'error'
    if details == 'xshort':
        output_dict['% diff'] = '{0:.0f}% ({1})'.format(diff_perc, 'sig' if pval<0.05 else 'not sig')  
    elif details == 'xlong':
        output_dict['% diff'] = '{0:.1f}% [{1:.1f}%, {2:.1f}%], p={3:.3f}'.format(diff_perc, lower_CI_perc, higher_CI_perc, pval)         
    return output_dict 
def output_table(df, col_N, cols_X, cols_mean, cols_std, hue, ctr_name='Control', exp_name='variant', details='xshort', col_width=110, output='color-coded'):
    units = col_N.split('_')[0] if '_' in col_N else col_N 
    n_ctr = df[df[hue]==ctr_name][col_N].values[0]
    n_exp = df[df[hue]==exp_name][col_N].values[0] 
    
    table = pd.DataFrame(columns=['metric', ctr_name, exp_name, '% diff'])
    table = table.append({'metric': '{} (unit of analysis)'.format(units), ctr_name:n_ctr, exp_name:n_exp, '% diff': 'N/A'}, ignore_index=True)
    
    for col_X in cols_X:  
        x_ctr = df[df[hue]==ctr_name][col_X].values[0]
        x_exp = df[df[hue]==exp_name][col_X].values[0] 
        results = z_test_prop_agg(n_ctr, n_exp, x_ctr, x_exp)  
        out_dict = output_format(results, ctr_name, exp_name, 'proportion', details)
        out_dict['metric'] = '% of {} {}'.format(units, col_X.replace(units+'_', ''))
        table = table.append(out_dict, ignore_index=True)
    for col_mean, col_std in zip(cols_mean, cols_std): 
        mean_ctr = df[df[hue]==ctr_name][col_mean].values[0]
        mean_exp = df[df[hue]==exp_name][col_mean].values[0]
        std_ctr = df[df[hue]==ctr_name][col_std].values[0]
        std_exp = df[df[hue]==exp_name][col_std].values[0]  
        results = t_test_mean_agg(n_ctr, n_exp, mean_ctr, mean_exp, std_ctr, std_exp)  
        out_dict = output_format(results, ctr_name, exp_name, 'mean', details)
        out_dict['metric'] = 'AVG {} per {}'.format(col_mean.replace('mean_', ''), units[:-1])  # get rid of plural form
        table = table.append(out_dict, ignore_index=True)   
    table.set_index('metric', inplace=True)
    table.rename_axis(None, inplace=True)
    if details == 'xlong':
        if output == 'raw':
            return table
        elif output == 'color-coded':
            return table.style.apply(highlight_xlong, axis=1).\
                        set_properties(subset=['% diff'], **{'width': '220px'}).\
                        set_properties(subset=[ctr_name], **{'width': '{}px'.format(col_width)}).\
                        set_properties(subset=[exp_name], **{'width': '{}px'.format(col_width)})
    elif details == 'xshort':
        if output == 'raw':
            return table
        elif output == 'color-coded':
            return table.style.apply(highlight_xshort, axis=1).\
                        set_properties(subset=['% diff'], **{'width': '90px'}).\
                        set_properties(subset=[ctr_name], **{'width': '{}px'.format(col_width)}).\
                        set_properties(subset=[exp_name], **{'width': '{}px'.format(col_width)})#.\
                        #set_table_styles([dict(selector='th', props=[('text-align', 'left')] ) ])

def color_red_or_green(cell, details):
    if details == 'xshort':
        if isinstance(cell, str) and 'sig' in cell:
            if cell.split('%')[1].strip().strip('(').strip(')') == 'sig' and float(cell.split('%')[0]) < 0:
                color = 'red'
            elif cell.split('%')[1].strip().strip('(').strip(')') == 'sig' and float(cell.split('%')[0]) > 0:
                color = 'green'
            else:
                color = 'black'
        else: 
            color = 'black'
    elif details == 'xlong':
        if isinstance(cell, str) and 'p=' in cell:
            if float(cell.split('p=')[1]) < 0.05 and float(cell.split('%')[0]) < 0:
                color = 'red'
            elif float(cell.split('p=')[1]) < 0.05 and float(cell.split('%')[0]) > 0:
                color = 'green'
            else:
                color = 'black'
        else: 
            color = 'black'        
    return 'color: %s' % color

def compare_multiple_groups(df, col_N, hue, details='xshort', col_width=110, output='color-coded'):
    df = df.copy()
    col_N = col_N
    cols_X = []
    cols_mean = []
    cols_std = []
    for column in df.columns: 
        if column.startswith(col_N.split('_')[0] if '_' in col_N else col_N) and column != col_N:
            cols_X.append(column)
        elif column.startswith('mean'):
            cols_mean.append(column)
        elif column.startswith('std'):
            cols_std.append(column)
    #cols_X = sorted(cols_X)
    #cols_mean = sorted(cols_mean)
    #cols_std = sorted(cols_std)
    if len(cols_mean) != len(cols_std):
        print("Error! Mean and std columns are not matching!")
    for col_mean, col_std in zip(cols_mean, cols_std):
        if col_mean.replace('mean_','') != col_std.replace('std_', ''):
            print("Error! Mean and std columns are not matching!")
    groups = sorted(df[hue].value_counts().index.to_list())
    n = len(groups)
    comps = {}
    for i in range(n-1):
        for j in range(i+1, n):
            comp = output_table(df, col_N, cols_X, cols_mean, cols_std, hue, ctr_name=groups[i], exp_name=groups[j], 
                                                      details=details, col_width=col_width, output='raw')
            comp = comp.rename(columns={'% diff':'% diff ({} - {})'.format(groups[j], groups[i])})
            for col in comp.columns:
                comps[col] = comp[[col]]
    cols_all = list(comps.keys())
    cols_diff = [x for x in cols_all if x.startswith('%')]
    cols_val = list(set(cols_all) - set(cols_diff))
    cols_diff = sorted(cols_diff)
    cols_val = sorted(cols_val)
    
    comps_table = pd.DataFrame()
    for col in cols_val:
        comps_table = pd.concat([comps_table, comps[col]], axis=1)
    for col in cols_diff:
        comps_table = pd.concat([comps_table, comps[col]], axis=1)  
    return comps_table if output=='raw' else comps_table.style.applymap(color_red_or_green, details=details)

# Sanity check functions, including proportional test for total sample size, and invariant metrics sanity check

In [21]:
from scipy import stats

def sanity_check(df, col, test_group, p_expected=0.5, sig=0.001):
    n = df[col].sum(axis=0)
    p = min(df[col][1]/n, df[col][0]/n)
    
    z_score = (p-p_expected)/((p_expected*p_expected/n)**0.5)
    pval = stats.norm.sf(abs(z_score))*2
    
    ci_lower = p - ((p_expected*p_expected/n)**0.5)*stats.norm.ppf((1-sig/2))  # 0.01 significance level 
    ci_higher = p + ((p_expected*p_expected/n)**0.5)*stats.norm.ppf((1-sig/2))  # 0.01 significance level 
    return pval, ci_lower, ci_higher, p 

def color3_red_or_green(cell):
    if isinstance(cell, str) and 'p=' in cell:
        if cell.endswith('fail'):
            color = 'red'
        else:
            color = 'green'
    else: 
        color = 'black'        
    return 'color: %s' % color

def sanity_check_exp_rate(df, col_N, variants):   
    
   # display(users)
    units = col_N.split('_')[0] if '_' in col_N else col_N 
    
    pval, ci_lower, ci_higher, p = sanity_check(df, col_N, variants)
    
    output = compare_multiple_groups(df, col_N, variants, details='xlong', col_width=110, output='raw')
    
    
    output['sanity check'] = 'Diff between buckets = ' + output.iloc[:,2] + \
                            np.where(output.iloc[:,2].str.split('p=').str[1].astype(float) <0.01, ', fail', ', pass')

    output['sanity check']['{} (unit of analysis)'.format(units)] = 'Bucket 1 porportion = {0:.1f}% [{1:.1f}%, {2:.1f}%], p={3:.8f}, {4}'.\
                            format(p*100, ci_lower*100, ci_higher*100, pval.round(8), 'fail' if pval<0.001 else 'pass')

    return output.iloc[:,[0,1,3]].style.applymap(color3_red_or_green) 

# Sanity check example
### Step 1. Use SQL to get sample sizes in test variants

In [11]:
# Get the sample sizes and invaariant metrics 
sanity_check_data = impala_run('''

WITH visitors_in_experiment AS
(
    SELECT
        REGEXP_EXTRACT(sa.test_set_session, 'browse_discussions_show_activity_links=([_A-Za-z0-9]+)', 1) AS test_group,
        visitor_id,
        MIN(dt) AS dt
    FROM l2.session_analytics sa
    WHERE dt BETWEEN '2020-04-22' AND '2020-06-01'
    AND user_id >0
    AND (REGEXP_EXTRACT(sa.test_set_session, 'browse_discussions_show_activity_links=([_A-Za-z0-9]+)', 1) IS NOT NULL AND REGEXP_EXTRACT(sa.test_set_session, 'browse_discussions_show_activity_links=([_A-Za-z0-9]+)', 1) != '')
    GROUP BY 1,2
),
    visitors_invariant AS 
(
    SELECT 
        u.test_group, 
        u.visitor_id, 
        COALESCE(SUM(sa.lb_view_photo),0) AS pageviews_viewphoto_lb, 
        COALESCE(SUM(sa.view_photo),0) AS pageviews_viewphoto 
    FROM visitors_in_experiment u 
    LEFT JOIN l2.session_analytics sa 
    ON u.visitor_id = sa.visitor_id 
    AND sa.dt BETWEEN '2020-03-22' AND '2020-04-20'  -- here it's an arbitary time windoe prior to the experiment start date
    GROUP BY 1,2 
) 
SELECT 
    test_group, 
    COUNT(visitor_id) AS visitors, 
    COUNT(CASE WHEN pageviews_viewphoto_lb>0 THEN visitor_id ELSE NULL END) AS visitors_had_photoview_lb_b4_test,
    AVG(pageviews_viewphoto_lb) AS mean_pageviews_viewphoto_lb_b4_test, 
    STDDEV(pageviews_viewphoto_lb) AS std_pageviews_viewphoto_lb_b4_test, 
    COUNT(CASE WHEN pageviews_viewphoto>0 THEN visitor_id ELSE NULL END) AS visitors_had_photoview_b4_test,
    AVG(pageviews_viewphoto) AS mean_pageviews_viewphoto_b4_test, 
    STDDEV(pageviews_viewphoto) AS std_pageviews_viewphoto_b4_test
FROM visitors_invariant
GROUP BY 1;''')

In [10]:
sanity_check_data

,test_group,visitors,visitors_had_photoview_lb_b4_test,mean_pageviews_viewphoto_lb_b4_test,std_pageviews_viewphoto_lb_b4_test,visitors_had_photoview_b4_test,mean_pageviews_viewphoto_b4_test,std_pageviews_viewphoto_b4_test
0,on,28908,5825,21.147572,109.036441,4709,4.663968,28.966770
1,off,28532,5714,21.347119,114.512796,4631,4.558285,29.231408


### Step 2. Call sanity check function 

In [22]:
# CALL the function for sanity check 
sanity_check_exp_rate(sanity_check_data, 'visitors', 'test_group')

,off,on,sanity check
visitors (unit of analysis),28532,28908,"Bucket 1 porportion = 49.7% [49.0%, 50.4%], p=0.11668356, pass"
% of visitors had_photoview_lb_b4_test,20.027%,20.150%,"Diff between buckets = 0.6% [-2.7%, 3.9%], p=0.712, pass"
% of visitors had_photoview_b4_test,16.231%,16.290%,"Diff between buckets = 0.4% [-3.4%, 4.1%], p=0.849, pass"
AVG pageviews_viewphoto_lb_b4_test per visitor,21.3471,21.1476,"Diff between buckets = -0.9% [-9.5%, 7.6%], p=0.831, pass"
AVG pageviews_viewphoto_b4_test per visitor,4.5583,4.6640,"Diff between buckets = 2.3% [-8.1%, 12.8%], p=0.663, pass"


# Sample size proportional test with user input segments 

In [24]:
def sanity_seg(test_name, 
               unit_of_randomization, 
               col_list, 
               variant_ctr, 
               variant_trm, 
               dt_analysis_start, 
               dt_analysis_end, 
               dt_test_first_day, 
               output='colorcoded'):
    data = impala_alpha_run('''
    WITH users0 AS
(
    SELECT
        sa.test_group,
        sa.{1}_id,
        sa.session_id,
        sa.landing_page_class,
        sa.page_type,
        pv.page_behavior,
        sa.landing_page_url, 
        sa.exit_page_url, 
        sa.exit_page_class,
        sa.medium, 
        sa.source, 
        sa.referer,
        sa.device_cat, 
        sa.browser, 
        sa.os, 
        sa.dt
    FROM
    (
        SELECT
            REGEXP_EXTRACT(test_set_session, '{0}=([_A-Za-z0-9]+)', 1) AS test_group,
            {1}_id,
            ROW_NUMBER() OVER (PARTITION BY {1}_id ORDER BY dt, start_ts) AS rn,
            signin_status,
            landing_page_class,
            landing_page_key,
            landing_page_url, 
            exit_page_url, 
            exit_page_class, 
            device_cat, 
            os, 
            browser, 
            medium, 
            referer, 
            source, 
            REGEXP_EXTRACT(LOWER(landing_page_url), 'houzz\.[a-z.]+\/([a-z_]+)', 1)  AS page_type,
            dt,
            session_id
        FROM l2.session_analytics sa
        WHERE sa.site_id = 101
        AND (sa.{1}_id IS NOT NULL AND CAST(sa.{1}_id AS STRING) != '0')
        AND (REGEXP_EXTRACT(sa.test_set_session, '{0}=([_A-Za-z0-9]+)', 1) IS NOT NULL AND REGEXP_EXTRACT(sa.test_set_session, '{0}=([_A-Za-z0-9]+)', 1) != '')
        AND sa.dt >= '{7}' AND sa.dt <= '{6}'
        AND signin_status = 'ALWAYS_SIGNED_IN'
    ) sa
    LEFT JOIN l2.page_views_daily pv
    ON sa.dt = pv.dt
    AND sa.session_id = pv.session_id
    AND sa.landing_page_key = pv.page_key
    WHERE sa.dt >= '{5}' AND sa.dt <= '{6}'
    AND sa.test_group IN ('{3}', '{4}')
    AND sa.rn = 1
)
SELECT
    {2}, 
    COUNT(CASE WHEN test_group = '{4}' THEN {1}_id ELSE NULL END) AS users_treatment,
    COUNT(CASE WHEN test_group = '{3}' THEN {1}_id ELSE NULL END) AS users_control
FROM users0
--WHERE medium = 'HOUZZ_OWN'
GROUP BY {2}; '''.format(test_name, unit_of_randomization, col_list, variant_ctr, variant_trm, dt_analysis_start, dt_analysis_end, dt_test_first_day))
    
    data['seg'] = ''
    cols = [x.strip() for x in col_list.split(',')] 
    for col in cols: 
        data['seg'] = data['seg']+', '+data[col]
    
    data_temp = data[['users_treatment', 'users_control', 'seg']].set_index('seg').T.reset_index()
    
    data['sanity check'] = ''
    for idx in data['seg']:
        pval, ci_lower, ci_higher, p = sanity_check(data_temp, idx, 'index')
        data.loc[data['seg']==idx, 'sanity check'] = 'Bucket 1 porportion = {0:.1f}% [{1:.1f}%, {2:.1f}%], p={3:.8f}, {4}'.\
                                format(p*100, ci_lower*100, ci_higher*100, pval.round(8), 'fail' if pval<0.001 else 'pass')
    
    data = data[cols + ['users_treatment', 'users_control', 'sanity check']].sort_values('users_treatment', ascending=False)
    if output == 'colorcoded':
        return data.style.applymap(color3_red_or_green) 
    elif output == 'raw':
        return data
    else:
        return ('Error! Please choose output = "colorcoded" or output = "raw"!')

# Example to call the function for sample size proportional tests of segments

In [25]:
sanity_seg('PRO_CONTACT_BUTTON_2', 'visitor', 'device_cat', 'Control', 'variant', '2019-08-15', '2019-10-07', '2019-08-10')

,device_cat,users_treatment,users_control,sanity check
0,Personal computer,33780,34577,"Bucket 1 porportion = 49.4% [48.8%, 50.0%], p=0.00230091, pass"
1,Smartphone,8421,8487,"Bucket 1 porportion = 49.8% [48.5%, 51.1%], p=0.61175340, pass"
2,Tablet,3320,3442,"Bucket 1 porportion = 49.1% [47.1%, 51.1%], p=0.13791015, pass"
3,Smart TV,1,0,"Bucket 1 porportion = 0.0% [-164.5%, 164.5%], p=0.31731051, pass"
